In [1]:
#UNDER CONSTRUCTION

import os
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import pandas_datareader
import matplotlib.pyplot as plt
import datetime
import tweepy
import twitterAPI
import sklearn
import pprint
import requests
from bs4 import BeautifulSoup
import json
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as vaderSA
import tqdm

pp = pprint.PrettyPrinter(indent=4)

path = "/Users/Matzekoek/Documents/Webhose.io - World News Articles sep-oct 2015/Webhose.io - World News Articles Sep 2015(2)"
os.chdir(path)

alphaVantageKey = '6AEOHIB29K740TG5'
twitterapikey = 'jNjZ10xjmy3f3pMTojP9kjNWY'
twitterapisecret = 'szou2LYaqQPXE9XMEKYnA3ETz2NZoHZcHuylswb7mgJxXCWPIE'
twitteraccesstoken = '1126856371989884929-m4K4b5Bm40yzIkHTr714a2wMRF5WaZ'
twitteraccesstokensecret = '49P9ZAs2w42IqJ059Fl5ylVvinnuF4wIQPP12xnDxxSHW'

ModuleNotFoundError: No module named 'twitterAPI'

In [ ]:
#Program design:


## NEWS

#access Twitter and download all FB tweets
#store news with headline, content, date, category(, no of newssources?, virality?)
#clean news data (universal time, correct for stock market closing, correct for weekends, remove unwanted characters from text)
#analyze sentiment of news content

## STOCK DATA

#access EOD stock data for FB stock from NYSE
#for specific time/date, predict if up/down on basis of sum of valence for that day
#(or weighted by # of sources/category/virality etc)

## IDEAS
#integrate NY weather data
#query twitter for keywords from headlines, and analyze sentiment
#widely covered, very viral, very negative news data can have negative effect on stock market (terror attacks, etc)


In [ ]:
#TO DO:

#access intraday stock data for NYSE
#for specific time/date, predict if up/down on basis of sum of valence for that hour/10min 
#(or weighted by # of sources/category/virality etc)

#DONE:

#Get news data
#store news with headline, content, date, category, no of newssources?, virality?
#clean news data (universal time, correct for news after trading closes, remove unwanted characters from text)
#analyze sentiment of content
#extract timezone, check for abberations



In [2]:
# twitterapikey = 
# twitterapisecret = 
# twitteraccesstoken = 
# twitteraccesstokensecret =  

CONSUMER_KEY = 'jNjZ10xjmy3f3pMTojP9kjNWY'
CONSUMER_SECRET = 'szou2LYaqQPXE9XMEKYnA3ETz2NZoHZcHuylswb7mgJxXCWPIE'
ACCESS_KEY = '1126856371989884929-m4K4b5Bm40yzIkHTr714a2wMRF5WaZ'
ACCESS_SECRET = '49P9ZAs2w42IqJ059Fl5ylVvinnuF4wIQPP12xnDxxSHW'

auth = tweepy.auth.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
api = tweepy.API(auth)
search_results = api.search(q="facebook", count=100)

In [3]:
# dflist = []

# for i in search_results:
#     dflist.append(i)

# pp.pprint(search_results.keys())

In [168]:
newsdf = pd.read_csv('Webhose.io - World News Articles Sep 2015.csv')

#newsdf.to_csv('Webhose.io - World News Articles Sep 2015.csv')

In [107]:
#prepare files for processing

files = []
for filename in os.listdir(path):
    files.append(str(filename))

In [108]:
#Get relevant news features

def getNewsFeatures(functionfile):
    
    jsonfile = json.load(open(str(functionfile)))
    
    date =           jsonfile['published']
    title =          jsonfile['title']
    source =         jsonfile['thread']['site']
    source_section = jsonfile['thread']['section_title']
    language =       jsonfile['language']
    country =        jsonfile['thread']['country']
    text =           jsonfile['text']
    return [date, title, source, source_section, language, country, text]
    #pp.print (date, title, source, source_section, language, country, text)

#samplefile = 'news_0000001.json'
#samplejson = json.load(open('news_0000001.json'))
#pp.pprint(getNewsFeatures(samplefile))

In [4]:
#Convert raw data to dataframe

columns = ['date', 'title', 'source','source_section','language','country','text']

newsdf = pd.DataFrame(columns = columns)

for file in files:
    
    jsondf = pd.DataFrame(getNewsFeatures(file)).T
    jsondf.columns = columns
    newsdf = newsdf.append(jsondf)
    

newsdf.reset_index(inplace = True) 

In [170]:
#correct news timestamp for: timezone, published after closed trading, weekend

newsdf['datetime'] = pd.to_datetime(newsdf['date'].str[0:19], format='%Y-%m-%dT%H:%M:%S')

utcdiff = datetime.timedelta(hours=-7)
newsdf['datetime_utc_corrected'] = newsdf['datetime'] + utcdiff
newsdf['date_1700_utc'] = pd.to_datetime(newsdf['datetime_utc_corrected'].astype(str).str[0:10] + ' ' + '17:00:00', format='%Y-%m-%d %H:%M:%S')

daydelta = datetime.timedelta(days = 1)
mask = newsdf.datetime_utc_corrected > newsdf.date_1700_utc
column_name = 'datetime_utc_corrected'
newsdf.loc[mask, column_name] = (newsdf['datetime_utc_corrected'].apply(lambda dt: dt.replace(hour=9, minute=0)) + daydelta)

twodaydelta = datetime.timedelta(days=2)
newsdf['date_weekday'] = newsdf.datetime_utc_corrected.apply(lambda dt: dt.weekday())

mask = newsdf.date_weekday == 5
column_name = 'datetime_utc_corrected'
newsdf.loc[mask, column_name] = (newsdf['datetime_utc_corrected'] + twodaydelta)

mask = newsdf.date_weekday == 6
column_name = 'datetime_utc_corrected'
newsdf.loc[mask, column_name] = (newsdf['datetime_utc_corrected'] + daydelta)


newsdf['text'] = newsdf['text'].str.replace(r'\n', ' ')
newsdf['text'] = newsdf['text'].str.replace("'", ' ')
newsdf['text'] = newsdf['text'].str.replace('"', ' ')
newsdf['text'] = newsdf['text'].str.replace('  ', ' ')

In [397]:
#Extract compound sentiment score with VADER, create sentiment dataframe, add to newsdf

analyser = vaderSA()

textlist = newsdf['text'].tolist()
sentimentdf = pd.DataFrame(columns = ['text_compound_sentiment_score'])

sentimentlist = []
for text in textlist:
    sentimentlist.append(analyser.polarity_scores(str(text))['compound'])
    
sentimentdf = pd.DataFrame(sentimentlist, columns = ['text_compound_sentiment'])

newsdf['text_compound_sentiment'] = sentimentdf['text_compound_sentiment']

In [235]:
# Get Finance Data from Alpha Vantage

#ts = TimeSeries(key='alphaVantageKey', output_format='pandas')
#stockdf, meta_data = ts.get_daily_adjusted(symbol='FB', outputsize='full')

#stockdf[(stockdf['date']>startdate) & (stockdf['date']<enddate)]

# print (data)
# data['5. adjusted close'].plot()
# plt.title('Stock chart')
# plt.show()

startdate = '2015-09-01'
enddate = '2015-09-30'

stockdf = stockdf[startdate:enddate]

stockdf['up'] = stockdf['5. adjusted close'] > stockdf['1. open']
stockdf['down'] = stockdf['5. adjusted close'] < stockdf['1. open']



# print (stockdf)
# stockdf['5. adjusted close'].plot()
# plt.title('Stock chart')
# plt.show()



In [ ]:
'''

getNewsFeatures(samplefile)


path = "/Users/Matzekoek/Documents/Webhose.io - World News Articles sep-oct 2015/Webhose.io - World News Articles Sep 2015"
os.chdir(path)

columns = ['date', 'title', 'source','source_section','language','country','text', 'sentiment']

newsdf = pd.DataFrame(columns = columns)


textlist = newsdf['text'].tolist()
sentimentdf = pd.DataFrame(columns = ['text_compound_sentiment_score'])

sentimentlist = []
for text in textlist:
    sentimentlist.append(analyser.polarity_scores(str(text))['compound'])

os.getcwd()

pp.pprint(samplejson['published'])
pp.pprint(samplejson['title'])
pp.pprint(samplejson['thread']['site'])
pp.pprint(samplejson['thread']['section_title'])
pp.pprint(samplejson['language'])
pp.pprint(samplejson['thread']['country'])
pp.pprint(samplejson['text'])


#s = pd.Series(getNewsFeatures(file), index=df.columns)
    
testdf = pd.DataFrame(getNewsFeatures(samplefile)).T
testdf.columns = columns

samplefile2 = 'news_0000002.json'

testdf2 = pd.DataFrame(getNewsFeatures(samplefile2)).T
testdf2.columns = columns

testdf = testdf.append(testdf2)
testdf.head()  


    
    
# year = str(newsdf.iloc[0:1]['date'])[5:9]
# month = str(newsdf.iloc[0:1]['date'])[11:12]
# day = str(newsdf.iloc[0:1]['date'])[13:15]
# hour = str(newsdf.iloc[0:1]['date'])[16:18]
# minute = str(newsdf.iloc[0:1]['date'])[19:21]
# secs = str(newsdf.iloc[0:1]['date'])[22:24]

#date = str(newsdf.iloc[0:1]['date'])[5:15]
#time = str(newsdf.iloc[0:1]['date'])[16:24]

#newsdf['singledate'] = str(newsdf.iloc[0:1]['date'])[5:15]
# FOUT   newsdf['time'] = (str(newsdf['date'])[16:24])

print(newsdf['datetime'].iloc[0:1].item())
utcdif = datetime.timedelta(hours=-5)

print(newsdf['datetime'].iloc[0:1].item() + utcdif)


#print(file)
#soup = BeautifulSoup(open(str(os.path.join(path,file))))

#newsdf['date_extract'] = newsdf['date'].str[0:10]
#newsdf['time_extract'] = newsdf['date'].str[11:19]
#newsdf['datetime_string'] = newsdf['date_extract'] + ' ' + newsdf['time_extract']

twodaydelta = datetime.timedelta(days=2)
newsdf['date_weekday'] = newsdf.datetime_utc_corrected.apply(lambda dt: dt.weekday())

mask = newsdf.date_weekday == 5
column_name = 'datetime_utc_corrected'
newsdf.loc[mask, column_name] = (newsdf['datetime_utc_corrected'] + twodaydelta)
                                 
mask = newsdf.date_weekday == 6
column_name = 'datetime_utc_corrected'
newsdf.loc[mask, column_name] = (newsdf['datetime_utc_corrected'] + daydelta)


newsdf[newsdf['datetime_utc_corrected'] > newsdf['date_1700_utc']]['datetime_utc_corrected'] = 

# twodaydelta = datetime.timedelta(days=2)
# newsdf['date_weekday'] = newsdf.datetime_utc_corrected.weekday()

#newsdf['datetime_1700'] = pd.to_datetime((newsdf['date_extract'] + ' ' + '17:00:00'), format='%Y-%m-%d %H:%M:%S')

#newsdf['utc_difference_extract'] = newsdf['date'].str[23:29]

#newsdf['text_compound_sentiment'] = analyser.polarity_scores(str(newsdf['text']))['compound']
# werkt niet, geeft waarde voor hele textbody van alles samen, zelfde als analyser.polarity_scores(str(newsdf['text'].iloc[:]))
#newsdf['text_compound_sentiment'] = newsdf['text'].apply(analyser.polarity_scores())
# for text in newsdf['text']:
#     newsdf['text_compound_sentiment'] = analyser.polarity_scores(text)

#checked with newsdf['utc_difference_extract'].value_counts(), all from same timestamp

#newsdf.to_csv('Webhose.io - World News Articles Sep 2015.csv')

#fbstockdata = quandl.get("EOD/FB", start_date=startdate, end_date=enddate)

'''